In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
tf.test.is_gpu_available()

True

In [3]:
numrows = 1000000
data = pd.read_csv("onehot2018_10k.csv", usecols=[i for i in range(1, 163)], dtype=int, chunksize=numrows)

In [4]:
data

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.regularizers import l1, l2

In [11]:
model = Sequential()
# model.add(LSTM(162, activation='tanh', return_sequences=True, input_shape=(256, 162), kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))
# model.add(LSTM(162, activation='tanh', return_sequences=True, kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))
# model.add(LSTM(162, activation='tanh', kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))

model.add(LSTM(256, return_sequences=True, input_shape=(512, 162)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(162, activation='sigmoid'))

opt = RMSprop(lr=1e-5, clipnorm=1.0)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 512, 256)          429056    
_________________________________________________________________
lstm_9 (LSTM)                (None, 512, 256)          525312    
_________________________________________________________________
lstm_10 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_12 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)              

In [12]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [13]:
for chunk in data:
    data_gen = TimeseriesGenerator(chunk[1:].values, chunk[:-1].values, length=512, stride=32, batch_size=64, shuffle=True)
    history = model.fit_generator(data_gen, epochs=2, verbose=1)
   


Epoch 1/2
489/489 [==============================] - 64s 131ms/step - loss: 62.3088 - accuracy: 0.0600
Epoch 2/2
489/489 [==============================] - 62s 127ms/step - loss: 58.1109 - accuracy: 0.0882
Epoch 1/2
489/489 [==============================] - 63s 128ms/step - loss: 64.8520 - accuracy: 0.0975
Epoch 2/2
 95/489 [====>.........................] - ETA: 50s - loss: 92.9147 - accuracy: 0.1036

KeyboardInterrupt: 

In [9]:
model.save("onehot2018_10k_take2.h5")

In [26]:
model.save_weights("onehot2018_10k_weights2")

In [32]:
del model

In [33]:
model = Sequential()
# model.add(LSTM(162, activation='tanh', return_sequences=True, input_shape=(256, 162), kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))
# model.add(LSTM(162, activation='tanh', return_sequences=True, kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))
# model.add(LSTM(162, activation='tanh', kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))

model.add(LSTM(162, return_sequences=True, input_shape=(512, 162)))
model.add(LSTM(162))
model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())
model.add(Dense(162, activation='sigmoid'))

opt = RMSprop(lr=0.000001, clipnorm=1.0)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 512, 162)          210600    
_________________________________________________________________
lstm_7 (LSTM)                (None, 162)               210600    
_________________________________________________________________
dense_12 (Dense)             (None, 512)               83456     
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_14 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_15 (Dense)             (None, 162)               41634     
Total params: 743,410
Trainable params: 743,410
Non-trainable params: 0
________________________________________________

In [34]:
model.load_weights("onehot2018_10k_weights")

In [35]:
data = pd.read_csv("onehot2018_20k.csv", usecols=[i for i in range(1, 163)], dtype=int, chunksize=numrows)

In [38]:
for chunk in data:
    data_gen = TimeseriesGenerator(chunk[1:].values, chunk[:-1].values, length=512, stride=32, batch_size=128, shuffle=True)
    history = model.fit_generator(data_gen, epochs=1, verbose=1)

61/61 [==============================] - 6s 94ms/step - loss: 43.7278 - accuracy: 0.3896


In [39]:
# model.save("onehot2018_20k_better.h5")

In [40]:
# model.save_weights("onehot2018_20k_weights.h5")

In [41]:
del model

In [42]:
from tensorflow.keras.models import load_model
model = tf.keras.models.load_model("onehot2018_10k_better.h5")

In [43]:
model = Sequential()
# model.add(LSTM(162, activation='tanh', return_sequences=True, input_shape=(256, 162), kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))
# model.add(LSTM(162, activation='tanh', return_sequences=True, kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))
# model.add(LSTM(162, activation='tanh', kernel_regularizer = l2(0.001), activity_regularizer=l2(0.001)))

model.add(LSTM(162, return_sequences=True, input_shape=(512, 162)))
model.add(LSTM(162))
model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())
model.add(Dense(162, activation='sigmoid'))

opt = RMSprop(lr=0.0000005, clipnorm=1.0)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 512, 162)          210600    
_________________________________________________________________
lstm_9 (LSTM)                (None, 162)               210600    
_________________________________________________________________
dense_16 (Dense)             (None, 512)               83456     
_________________________________________________________________
dense_17 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_18 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_19 (Dense)             (None, 162)               41634     
Total params: 743,410
Trainable params: 743,410
Non-trainable params: 0
________________________________________________

In [44]:
model.load_weights("onehot2018_20k_weights.h5")

In [45]:
data = pd.read_csv("onehot2018_30k.csv", usecols=[i for i in range(1, 163)], dtype=int, chunksize=numrows)

In [ ]:
for chunk in data:
    data_gen = TimeseriesGenerator(chunk[1:].values, chunk[:-1].values, length=512, stride=32, batch_size=128, shuffle=True)
    history = model.fit_generator(data_gen, epochs=1, verbose=1)

245/245 [==============================] - 23s 92ms/step - loss: 51.8160 - accuracy: 0.3876


In [ ]:
model.save("onehot2018_30k_better.h5")

In [ ]:
model.save_weights("onehot2018_30k_weights.h5")